In [ ]:
import json
import socket
import time
import urllib.request
from pprint import pprint
from elasticsearch import Elasticsearch, helpers
import json


In [ ]:
while True:
    response = urllib.request.urlopen('https://feeds.divvybikes.com/stations/stations.json')

    # Extract the body of the reply
    response_body = response.read()

    # Decode the format in json format
    stations_json = json.loads(response_body.decode("utf-8"))

    stations_json = stations_json['stationBeanList']

    for item in stations_json:
            if item["city"] == "Chicago ":
                item["city"] = "Chicago"
                
    list_divvy_documents = []           
    
    for stations in stations_json:
        index_stations = {  
                "_index": "divvy_stations_logs1",
                "_type": "logs",
                "_source": stations
        }
        list_divvy_documents.append(index_stations)  
    
    output_file = open('divvyToES.json', 'w', encoding='utf-8')

    for dic in list_divvy_documents:
        json.dump(dic, output_file) 
        output_file.write("\n")
    
    
    es1 = Elasticsearch()
    
    helpers.bulk(es1, list_divvy_documents)
    
    # Sleep for 125 seconds; divvy updates its stations status every 2 minutes
    print('Sent Heartbeat to Divvy Servers and Going to sleep for 125 seconds now ...')
    time.sleep(125)    
    continue